In [2]:
from fastai.vision import *

In [3]:
import pandas as pd
from pathlib import Path

In [4]:
TRAIN_PATH = Path("/home/shaun/personal/kaggle-data/nybg2020/train/")

In [5]:
MODEL_PATH = Path("/home/shaun/personal/kaggle-data/nybg2020/models/")

In [5]:
df = pd.read_csv(TRAIN_PATH/"train.csv")

In [15]:
classes = df["category_id"].value_counts()

In [14]:
16425 /len(df["category_id"].value_counts())   

0.5117938491259776

In [8]:
len(df)/32

32210.84375

In [9]:
641860/32

20058.125

In [8]:
df_train = df[[ "file_name", "category_id"]]
df_train =  df_train.rename(columns={"category_id": "label", "file_name":"name"})


In [9]:
df_train.head()

,name,label
0,images/156/72/354106.jpg,15672
1,images/156/72/545181.jpg,15672
2,images/156/72/449419.jpg,15672
3,images/156/72/200223.jpg,15672
4,images/156/72/5327.jpg,15672


In [10]:
sample = df_train.groupby('label').apply(lambda x: x.sample(n=20, replace=True)).reset_index(drop = True)

In [11]:
sample.head()

,name,label
0,images/000/00/437000.jpg,0
1,images/000/00/897248.jpg,0
2,images/000/00/72077.jpg,0
3,images/000/00/897248.jpg,0
4,images/000/00/818271.jpg,0


In [12]:
len(sample)

641860

In [13]:
data = (ImageList.from_df(df_train,TRAIN_PATH)
        .split_by_rand_pct()
        .label_from_df()
        .transform(size=128)
        .databunch(bs=128))

/home/shaun/personal/.pytorch-env/lib/python3.6/site-packages/fastai/data_block.py:534: UserWarning: You are labelling your items with CategoryList.
Your valid set contained the following unknown labels, the corresponding items have been discarded.
22564, 18719, 11851, 18282, 16992...
  if getattr(ds, 'warn', False): warn(ds.warn)


In [14]:
#data.show_batch()

In [15]:
learn = cnn_learner(data, models.resnet18, metrics=accuracy)

In [45]:
#learn.unfreeze()

In [46]:
#learn.fit_one_cycle(3, max_lr=(1e-4, 1e-3, 1e-2), wd=(1e-4,1e-4,1e-1))

epoch,train_loss,valid_loss,accuracy,time
0,6.721348,6.331872,0.168288,1:08:43
1,5.734519,4.950062,0.324112,1:10:00
2,4.954755,4.332292,0.452582,1:09:44


In [19]:
learn.save(MODEL_PATH/"model-3")

In [36]:
#! ls $MODEL_PATH

model-1.pth


## Inference

In [23]:
# load model
learn.load(MODEL_PATH/"model-2")

RuntimeError: Error(s) in loading state_dict for Sequential:
	size mismatch for 1.8.weight: copying a param with shape torch.Size([31895, 512]) from checkpoint, the shape in current model is torch.Size([31942, 512]).
	size mismatch for 1.8.bias: copying a param with shape torch.Size([31895]) from checkpoint, the shape in current model is torch.Size([31942]).

In [1]:
import json


In [ ]:
# get test data
TEST_PATH = Path("/home/shaun/personal/kaggle-data/nybg2020/test/")
with open(TEST_PATH/"metadata.json", encoding="utf8", errors='ignore') as json_file:
    data = json.load(json_file)
df_test =  pd.DataFrame.from_dict(data["images"])

In [ ]:
# inference
from fastai.vision import ImageImageList
test_data = ImageImageList.from_df(df_test, TEST_PATH/'images', cols="file_name")
learn.data.add_test(test_data)
preds, y = learn.get_preds(ds_type=DatasetType.Test)